In [1]:
# Define sub-periods
periods = {
    'Period 1': ('2007-03-01', '2007-06-30'),
    'Period 2': ('2007-07-01', '2009-03-01'),
    'Period 3': ('2009-03-02', '2020-01-31'),
    'Period 4': ('2020-02-01', '2020-05-01'),
    'Period 5': ('2020-05-02', '2024-03-31')
}

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import cvxpy as cp
from scipy.optimize import minimize
from multiprocessing import Pool, cpu_count
from scipy.stats import skew, kurtosis

In [3]:
# Load ETF returns data
etf_returns = pd.read_csv('etf_prices.csv', index_col=0, parse_dates=True).pct_change().dropna()

# Load Fama-French factors data
ff_factors = pd.read_csv('./F-F_Research_Data_Factors_daily.CSV', skiprows=3, index_col=0)
ff_factors.index = pd.to_datetime(ff_factors.index, format='%Y%m%d', errors='coerce')
ff_factors = ff_factors.dropna()
ff_factors = ff_factors.loc['2007-03-01':'2024-03-31']

# Reindex ETF returns to match Fama-French factors dates
etf_returns = etf_returns.reindex(ff_factors.index).dropna()
returns_data = etf_returns.copy()

# Define beta constraints and lambda
beta_constraints = [-2, 2]
lambd = 0.1

# Define the Fama-French factor loadings DataFrame
factor_loadings_df = pd.DataFrame({
    'Alpha': [0.02, 0.01, 0.015, 0.018, 0.016, 0.013, 0.014, 0.017, 0.011, 0.012, 0.013, 0.014],
    'Mkt-RF': [1.0, 1.2, 1.1, 1.3, 1.1, 1.0, 1.2, 1.3, 1.0, 1.2, 1.1, 1.3],
    'SMB': [0.3, 0.2, 0.25, 0.28, 0.26, 0.24, 0.25, 0.27, 0.22, 0.23, 0.25, 0.26],
    'HML': [0.2, 0.15, 0.18, 0.19, 0.17, 0.16, 0.18, 0.19, 0.15, 0.17, 0.18, 0.19]
})

factor_loadings_df.index = ['FXE', 'EWJ', 'GLD', 'QQQ', 'SPY', 'SHV', 'DBA', 'USO', 'XBI', 'ILF', 'EPP', 'FEZ']

In [4]:
# Tracking Error Volatility Function for Strategy II
def tracking_error_volatility(weights, returns_data, benchmark_returns):
    portfolio_returns = returns_data @ weights
    return np.sqrt(np.var(portfolio_returns - benchmark_returns))

# Optimize Strategy I
def optimize_strategy_I(expected_returns, cov_matrix, factor_loadings, beta_constraints, lambd):
    n = len(expected_returns)
    w = cp.Variable(n)
    portfolio_return = expected_returns @ w
    cov_matrix = (cov_matrix + cov_matrix.T) / 2
    portfolio_risk = cp.quad_form(w, cov_matrix)
    portfolio_beta = factor_loadings['Mkt-RF'].values @ w
    constraints = [cp.sum(w) == 1, portfolio_beta >= beta_constraints[0], portfolio_beta <= beta_constraints[1], w >= -2, w <= 2]
    objective = cp.Maximize(portfolio_return - lambd * cp.norm(portfolio_risk, 2))
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return w.value

# Optimize Strategy II
def optimize_strategy_II(expected_returns, returns_data, factor_loadings, beta_constraints, lambd, benchmark_returns):
    n = len(expected_returns)
    def objective(weights):
        portfolio_return = expected_returns @ weights
        te_vol = tracking_error_volatility(weights, returns_data, benchmark_returns)
        return -(portfolio_return - lambd * te_vol)
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}, {'type': 'ineq', 'fun': lambda w: beta_constraints[1] - np.sum(w * factor_loadings['Mkt-RF'].values)}, {'type': 'ineq', 'fun': lambda w: np.sum(w * factor_loadings['Mkt-RF'].values) - beta_constraints[0]}, {'type': 'ineq', 'fun': lambda w: 2 - w}, {'type': 'ineq', 'fun': lambda w: w + 2}]
    bounds = [(-2, 2) for _ in range(n)]
    result = minimize(objective, np.ones(n) / n, bounds=bounds, constraints=constraints)
    return result.x

In [5]:
def rolling_backtest(returns_data, ff_factors, beta_constraints, lambd, window_length, rebalancing_frequency):
    dates = returns_data.index
    portfolio_returns_I = []
    portfolio_returns_II = []
    benchmark_returns = returns_data['SPY'].values
    
    for i in range(window_length, len(dates), rebalancing_frequency):
        train_data = returns_data.iloc[i-window_length:i]
        test_data = returns_data.iloc[i:i+rebalancing_frequency]
        test_dates = test_data.index
        
        if len(test_data) < rebalancing_frequency:
            break
        
        # Strategy I
        weights_I = optimize_strategy_I(factor_loadings_df['Alpha'].values, train_data.cov().values, factor_loadings_df, beta_constraints, lambd)
        strat_returns_I = test_data @ weights_I
        portfolio_returns_I.extend(strat_returns_I)
        
        # Strategy II
        weights_II = optimize_strategy_II(factor_loadings_df['Alpha'].values, train_data, factor_loadings_df, beta_constraints, lambd, benchmark_returns[i-window_length:i])
        strat_returns_II = test_data @ weights_II
        portfolio_returns_II.extend(strat_returns_II)
        
    return np.array(portfolio_returns_I), np.array(portfolio_returns_II)

In [6]:
def backtest_period(period_data):
    period_name, (start_date, end_date) = period_data
    period_returns_data = returns_data.loc[start_date:end_date]
    period_ff_factors = ff_factors.loc[start_date:end_date]
    
    strategy_I_returns, strategy_II_returns = rolling_backtest(period_returns_data, period_ff_factors, beta_constraints, lambd, window_length, rebalancing_frequency)
    benchmark_returns = period_returns_data['SPY'].values[len(strategy_I_returns):]
    
    return period_name, strategy_I_returns, strategy_II_returns, benchmark_returns

# Define rolling window length and rebalancing frequency
window_length = 60  # 60 days for historical data
rebalancing_frequency = 5  # Weekly rebalancing (every 5 days)

# Run backtests in parallel
with Pool(cpu_count()) as pool:
    results = pool.map(backtest_period, periods.items())

# Convert results to dictionary format
results_dict = {}
for period_name, strategy_I_returns, strategy_II_returns, benchmark_returns in results:
    results_dict[period_name] = {
        'Strategy I': strategy_I_returns,
        'Strategy II': strategy_II_returns,
        'Benchmark': benchmark_returns
    }

In [ ]:
def calculate_kpis(returns):
    if len(returns) == 0:
        return {
            'Cumulative Return': np.nan,
            'Mean Return': np.nan,
            'Geometric Mean Return': np.nan,
            'Min Return': np.nan,
            'Max Drawdown': np.nan,
            'Sharpe Ratio': np.nan,
            'Volatility': np.nan,
            'Skewness': np.nan,
            'Kurtosis': np.nan,
            'VaR (5%)': np.nan,
            'CVaR (5%)': np.nan
        }
    
    cum_return = np.cumprod(1 + returns)[-1] - 1
    mean_return = np.mean(returns)
    geo_mean_return = np.exp(np.mean(np.log(1 + returns))) - 1
    min_return = np.min(returns)
    max_dd = np.max(np.maximum.accumulate(np.cumsum(returns)) - np.cumsum(returns))
    sharpe_ratio = np.mean(returns) / np.std(returns)
    vol = np.std(returns)
    skewness = skew(returns)
    kurt = kurtosis(returns)
    var = np.percentile(returns, 5)
    cvar = returns[returns <= var].mean()
    
    return {
        'Cumulative Return': cum_return,
        'Mean Return': mean_return,
        'Geometric Mean Return': geo_mean_return,
        'Min Return': min_return,
        'Max Drawdown': max_dd,
        'Sharpe Ratio': sharpe_ratio,
        'Volatility': vol,
        'Skewness': skewness,
        'Kurtosis': kurt,
        'VaR (5%)': var,
        'CVaR (5%)': cvar
    }

# Calculate KPIs for each period and strategy
kpi_results = {}
for period_name, period_results in results_dict.items():
    kpi_results[period_name] = {
        'Strategy I': calculate_kpis(period_results['Strategy I']),
        'Strategy II': calculate_kpis(period_results['Strategy II']),
        'Benchmark': calculate_kpis(period_results['Benchmark'])
    }

# Display KPIs in a table format
kpi_df = pd.DataFrame(kpi_results).T
print(kpi_df)